# Database Setup - Streamlined Version

## Purpose
Initialize FAQ tracking database with schema and test data.

## What This Does
1. Creates database schema (7 tables, 9 views, 1 trigger)
2. Loads test data from CSV files
3. Creates FAQ-content mappings
4. Verifies setup

## Test Data Source
- `../data/test_content_repo.csv`
- `../data/test_faq_questions.csv`
- `../data/test_faq_answers.csv`

In [1]:
# Setup
import sys
import sqlite3
import pandas as pd
from pathlib import Path
import json  # Added for v4 schema JSON metadata support

DATA_SOURCE = 'test'  # Options: 'excel' or 'test'

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / 'src'
DATABASE_DIR = PROJECT_ROOT / 'databases'
DATABASE_NAME = 'faq_update.db'
DB_PATH = DATABASE_DIR / DATABASE_NAME
SQL_DIR = PROJECT_ROOT / 'sql'
DOCS_DIR = PROJECT_ROOT / 'docs'
TEST_DATA_DIR = DOCS_DIR / 'test_sample_data'

# Schema file
SCHEMA_FILE = SQL_DIR / 'create_schema_v4.sql'

# Excel files (when DATA_SOURCE='excel')
CONTENT_REPO_XLSX = DOCS_DIR / 'content_repo.xlsx'
FAQ_QUESTION_XLSX = DOCS_DIR / 'faq_question.xlsx'
FAQ_ANSWER_XLSX = DOCS_DIR / 'faq_answer.xlsx'
sys.path.insert(0, str(SRC_DIR))


pd.set_option('display.max_columns', None)

print(f"📁 Database: {DB_PATH}")
DATABASE_DIR.mkdir(parents=True, exist_ok=True)

📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\faq_update.db


In [2]:
# Import utility functions from src/utils
from utils import (
    BASE_TIME,
    format_timestamp,
    get_timestamp,
    calculate_checksum,
    calculate_file_checksum,
    validate_checksum,
    create_markdown_file,
    compress_diff,
    decompress_diff,
    preview_dataframe,
    print_config_banner,
    log_success,
    log_info,
    log_warning,
    log_error
)

## Step 1: Create Schema

In [3]:
# Load and execute schema SQL
schema_file = PROJECT_ROOT / 'sql' / 'create_schema_v4.sql'

if not schema_file.exists():
    raise FileNotFoundError(f"Schema file not found: {schema_file}")

with open(schema_file, 'r', encoding='utf-8') as f:
    schema_sql = f.read()

with sqlite3.connect(DB_PATH) as conn:
    conn.executescript(schema_sql)
    
    # Verify
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)
    views = pd.read_sql("SELECT name FROM sqlite_master WHERE type='view' ORDER BY name", conn)
    
log_success(f"Created {len(tables)} tables and {len(views)} views")

✅ Created 8 tables and 9 views


## Step 2: Excel Loaders functions

In [4]:
import hashlib
from datetime import datetime, timedelta

# ============================================================================
# EXCEL DATA LOADERS
# ============================================================================

def load_content_repo_from_excel() -> pd.DataFrame:
    """
    Load content_repo from Excel file with validation.
    """
    log_info(f"Loading {CONTENT_REPO_XLSX.name}...")
    
    if not CONTENT_REPO_XLSX.exists():
        log_error(f"File not found: {CONTENT_REPO_XLSX}")
        raise FileNotFoundError(f"Excel file not found: {CONTENT_REPO_XLSX}")
    
    df = pd.read_excel(CONTENT_REPO_XLSX)
    log_success(f"Loaded {len(df)} records from Excel")
    
    return df


def load_faq_questions_from_excel() -> pd.DataFrame:
    """
    Load faq_questions from Excel with deduplication.
    """
    log_info(f"Loading {FAQ_QUESTION_XLSX.name}...")
    
    if not FAQ_QUESTION_XLSX.exists():
        log_error(f"File not found: {FAQ_QUESTION_XLSX}")
        raise FileNotFoundError(f"Excel file not found: {FAQ_QUESTION_XLSX}")
    
    df = pd.read_excel(FAQ_QUESTION_XLSX)
    log_success(f"Loaded {len(df)} records from Excel")

    return df


def load_faq_answers_from_excel() -> pd.DataFrame:
    """
    Load faq_answers from Excel.
    """
    log_info(f"Loading {FAQ_ANSWER_XLSX.name}...")
    
    if not FAQ_ANSWER_XLSX.exists():
        log_error(f"File not found: {FAQ_ANSWER_XLSX}")
        raise FileNotFoundError(f"Excel file not found: {FAQ_ANSWER_XLSX}")
    
    df = pd.read_excel(FAQ_ANSWER_XLSX)
    log_success(f"Loaded {len(df)} records from Excel")
    
    return df

log_success("Excel loader functions defined")

✅ Excel loader functions defined


## Step 3: Test Data Generators

In [5]:
def generate_test_content_repo() -> pd.DataFrame:
    """
    Generate comprehensive test data for content_repo with real markdown files.
    Creates 3 pages from Employee Leave Policy with checksums.
    """
    log_info("Generating test data for content_repo...")
    
    # Create markdown directory
    markdown_dir = TEST_DATA_DIR / 'markdown_files' / 'leave_policy'
    markdown_dir.mkdir(parents=True, exist_ok=True)
    
    # Policy content
    policy_content = {
        'page1_v1': """# Employee Leave Policy

## 1. Purpose
This policy establishes guidelines for employee leave requests and approvals.

## 2. Scope
This policy applies to all full-time employees.

## 3. Annual Leave
- Employees accrue 15 days of annual leave per year
- Leave must be requested at least 2 weeks in advance
- Maximum carryover: 5 days to next year
""",
        'page2_v1': """## 4. Sick Leave
Employees are entitled to sick leave as follows:
- 10 days of paid sick leave per year
- Medical certificate required for absences over 3 consecutive days
- Unused sick leave does not carry over

## 5. Parental Leave
- 12 weeks of paid parental leave for primary caregiver
- 4 weeks of paid parental leave for secondary caregiver
""",
        'page3_v1': """## 6. Bereavement Leave
- 5 days for immediate family member
- 2 days for extended family member
- No documentation required

## 7. Leave Request Process
1. Submit request via HR portal
2. Manager approval required
3. Confirmation sent within 48 hours
"""
    }
    
    # Create records
    records = []
    base_time = datetime(2025, 1, 1, 10, 0, 0)
    
    # Page 1
    md_path_p1 = markdown_dir / 'leave_policy' / 'page1_v1.md'
    checksum_p1 = create_markdown_file(md_path_p1, policy_content['page1_v1'])

    now = format_timestamp(use_base_time=False)
    
    records.append({
        'ud_source_file_id': 1001,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p1.absolute()),
        'extracted_layout_file_path': 'layout_p1.txt',
        'raw_file_page_nbr': 1,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p1,
        'file_status': 'Active',
        'created_dt': now,
        'last_modified_dt': now
    })
    
    # Page 2
    md_path_p2 = markdown_dir / 'leave_policy' / 'page2_v1.md'
    checksum_p2 = create_markdown_file(md_path_p2, policy_content['page2_v1'])
    
    records.append({
        'ud_source_file_id': 1002,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p2.absolute()),
        'extracted_layout_file_path': 'layout_p2.txt',
        'raw_file_page_nbr': 2,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p2,
        'file_status': 'Active',
        'created_dt': now,
        'last_modified_dt': now
    })
    
    # Page 3
    md_path_p3 = markdown_dir / 'leave_policy' / 'page3_v1.md'
    checksum_p3 = create_markdown_file(md_path_p3, policy_content['page3_v1'])
    
    records.append({
        'ud_source_file_id': 1003,
        'domain': 'HR',
        'service': 'Policy Management',
        'orgoid': 'ORG001',
        'associateoid': 'ASSOC001',
        'raw_file_nme': 'Employee_Leave_Policy.pdf',
        'raw_file_type': '.pdf',
        'raw_file_version_nbr': 1,
        'source_url_txt': None,
        'parent_location_txt': None,
        'raw_file_path': '/test/Employee_Leave_Policy.pdf',
        'extracted_markdown_file_path': str(md_path_p3.absolute()),
        'extracted_layout_file_path': 'layout_p3.txt',
        'raw_file_page_nbr': 3,
        'title_nme': 'Employee Leave Policy',
        'breadcrumb_txt': None,
        'content_tags_txt': None,
        'version_nbr': 1,
        'content_checksum': checksum_p3,
        'file_status': 'Active',
        'created_dt': now,
        'last_modified_dt': now
    })
    
    df = pd.DataFrame(records)
    log_success(f"Generated {len(df)} content_repo records with markdown files")
    return df

def generate_test_faq_questions() -> pd.DataFrame:
    """
    Generate 10 comprehensive test questions covering all 3 pages.
    """
    log_info("Generating test data for faq_questions...")

    now = format_timestamp(use_base_time=False)

    records = [
            # Page 1 questions (Annual Leave)
            {
                'question_id': 1001,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'What is the annual leave policy?',
                'question_txt': 'What is the annual leave policy?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1001,
                'src_page_number': 1,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1002,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'How many annual leave days do I get per year?',
                'question_txt': 'How many annual leave days do I get per year?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1001,
                'src_page_number': 1,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1003,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'Can I carry over unused annual leave to next year?',
                'question_txt': 'Can I carry over unused annual leave to next year?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1001,
                'src_page_number': 1,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            
            # Page 2 questions (Sick Leave & Parental Leave)
            {
                'question_id': 1004,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'How many sick leave days do employees get?',
                'question_txt': 'How many sick leave days do employees get?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1002,
                'src_page_number': 2,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1005,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'When do I need a medical certificate for sick leave?',
                'question_txt': 'When do I need a medical certificate for sick leave?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1002,
                'src_page_number': 2,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1006,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'What is the parental leave policy?',
                'question_txt': 'What is the parental leave policy?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1002,
                'src_page_number': 2,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1007,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'How much parental leave do primary caregivers get?',
                'question_txt': 'How much parental leave do primary caregivers get?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1002,
                'src_page_number': 2,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            
            # Page 3 questions (Bereavement & Leave Process)
            {
                'question_id': 1008,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'What is the bereavement leave policy?',
                'question_txt': 'What is the bereavement leave policy?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1003,
                'src_page_number': 3,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1009,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'How do I request leave?',
                'question_txt': 'How do I request leave?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1003,
                'src_page_number': 3,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            },
            {
                'question_id': 1010,
                'prev_recommended_question_txt': None,
                'prev_question_txt': None,
                'recommeded_question_txt': 'How long does it take to get leave approval?',
                'question_txt': 'How long does it take to get leave approval?',
                'source': 'test_data',
                'src_file_name': 'Employee_Leave_Policy.pdf',
                'src_id': 1003,
                'src_page_number': 3,
                'prev_src': None,
                'domain': 'HR',
                'service': 'Policy Management',
                'product': 'Leave Management',
                'orgid': 'ORG001',
                'created': now,
                'created_by': 'test_generator',
                'modified': now,
                'modified_by': 'test_generator',
                'version': 1,
                'status': 'Active',
                'metadata': '{}'
            }
        ]
        
    df = pd.DataFrame(records)
    log_success(f"Generated {len(df)} faq_questions")
    return df

def generate_test_faq_answers() -> pd.DataFrame:
    """
    Generate 10 comprehensive test answers matching questions.
    """
    log_info("Generating test data for faq_answers...")
    
    now = format_timestamp(use_base_time=False)
    
    records = [
        # Answers for Page 1 questions
        {
            'answer_id': 1001,
            'question_id': 1001,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees accrue 15 days of annual leave per year.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Employees accrue 15 days of annual leave per year. Leave must be requested at least 2 weeks in advance, and a maximum of 5 days can be carried over to the next year.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1002,
            'question_id': 1002,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees accrue 15 days annually.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'All full-time employees accrue 15 days of annual leave per year.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1003,
            'question_id': 1003,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Yes, up to 5 days can be carried over.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Yes, you can carry over unused annual leave to the next year, with a maximum carryover of 5 days.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1001',
            'src_page_number': 1,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Answers for Page 2 questions
        {
            'answer_id': 1004,
            'question_id': 1004,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Employees receive 10 days of paid sick leave per year.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Employees are entitled to 10 days of paid sick leave per year. A medical certificate is required for absences over 3 consecutive days. Unused sick leave does not carry over.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1005,
            'question_id': 1005,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Medical certificate required for absences over 3 consecutive days.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'A medical certificate is required for sick leave absences that exceed 3 consecutive days.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1006,
            'question_id': 1006,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '12 weeks paid for primary caregiver, 4 weeks for secondary.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Primary caregivers receive 12 weeks of paid parental leave, while secondary caregivers receive 4 weeks of paid parental leave.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1007,
            'question_id': 1007,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '12 weeks of paid parental leave.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Primary caregivers are entitled to 12 weeks of paid parental leave.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1002',
            'src_page_number': 2,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        
        # Answers for Page 3 questions
        {
            'answer_id': 1008,
            'question_id': 1008,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': '5 days for immediate family, 2 days for extended family.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Bereavement leave provides 5 days for immediate family members and 2 days for extended family members. No documentation is required.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1009,
            'question_id': 1009,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Submit via HR portal and get manager approval.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'To request leave: 1) Submit request via HR portal, 2) Obtain manager approval, 3) Receive confirmation within 48 hours.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        },
        {
            'answer_id': 1010,
            'question_id': 1010,
            'prev_recommended_answer_txt': None,
            'recommended_answer_txt': 'Confirmation is sent within 48 hours.',
            'prev_faq_answer_txt': None,
            'faq_answer_txt': 'Leave approval confirmation is sent within 48 hours of submission, after manager approval.',
            'user_feedback_txt': None,
            'source': 'test_data',
            'src_file_name': 'Employee_Leave_Policy.pdf',
            'src_id': '1003',
            'src_page_number': 3,
            'prev_src': None,
            'domain': 'HR',
            'service': 'Policy Management',
            'product': 'Leave Management',
            'orgid': 'ORG001',
            'created': now,
            'created_by': 'test_generator',
            'modified': now,
            'modified_by': 'test_generator',
            'version': 1,
            'status': 'Active',
            'metadata': '{}'
        }
    ]
    
    df = pd.DataFrame(records)
    log_success(f"Generated {len(df)} faq_answers")
    return df

log_success("Test data generator functions (base tables) defined")

✅ Test data generator functions (base tables) defined


## Step 4: Load Test Data

In [6]:
def load_all_data(source: str = DATA_SOURCE) -> dict:
    """
    Load all datasets based on configured source.
    
    Returns dict with all DataFrames needed for database population.
    """
    log_info(f"Loading data from {source.upper()} source...")
    
    data = {'source': source}
    
    if source == 'excel':
        # Load base tables from Excel
        data['content_repo'] = load_content_repo_from_excel()
        data['faq_questions'] = load_faq_questions_from_excel()
        data['faq_answers'] = load_faq_answers_from_excel()
        
        log_success("Excel data loaded (base tables only)")
        log_info("Tracking tables will remain empty (populate via pipeline)")
        
    elif source == 'test':
        # Generate complete test dataset
        data['content_repo'] = generate_test_content_repo()
        data['faq_questions'] = generate_test_faq_questions()
        data['faq_answers'] = generate_test_faq_answers()
        # data['content_change_log'] = generate_test_content_change_log(data['content_repo'])
        # data['faq_content_map'] = generate_test_faq_content_map(
        #     data['content_repo'],
        #     data['faq_questions'],
        #     data['faq_answers'],
        #     data['content_change_log']
        # )
        # data['content_diffs'] = generate_test_content_diffs(data['content_change_log'])
        
        # Count actual tables generated (excluding 'source' key)
        tables_generated = len([k for k in data.keys() if k != 'source'])
        log_success(f"Test data generated ({tables_generated} base tables)")
        log_info("Tracking tables (content_change_log, faq_content_map, content_diffs) commented out")
        log_info("These will be populated by pipeline during operation")
        
    else:
        raise ValueError(f"Invalid DATA_SOURCE: {source}. Must be 'excel' or 'test'")
    
    return data

# Execute unified loader
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

test_data = load_all_data()

# Display summary
print(f"\n📊 Data Summary (Source: {test_data['source'].upper()}):")
for table_name, df in test_data.items():
    if table_name != 'source' and df is not None:
        print(f"  - {table_name:25} {len(df):5} records")


LOADING DATA
ℹ️  Loading data from TEST source...
ℹ️  Generating test data for content_repo...
✅ Generated 3 content_repo records with markdown files
ℹ️  Generating test data for faq_questions...
✅ Generated 10 faq_questions
ℹ️  Generating test data for faq_answers...
✅ Generated 10 faq_answers
✅ Test data generated (3 base tables)
ℹ️  Tracking tables (content_change_log, faq_content_map, content_diffs) commented out
ℹ️  These will be populated by pipeline during operation

📊 Data Summary (Source: TEST):
  - content_repo                  3 records
  - faq_questions                10 records
  - faq_answers                  10 records


In [7]:
# # Load test data from CSV
# test_data = load_test_data()
# print(get_data_summary(test_data))

## Step 5: Insert Data

In [8]:
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Clear existing data
    for table in ['faq_content_map', 'content_change_log', 'faq_answers', 'faq_questions', 'content_repo']:
        conn.execute(f"DELETE FROM {table}")
    
    # Insert test data
    test_data['content_repo'].to_sql('content_repo', conn, if_exists='append', index=False)
    test_data['faq_questions'].to_sql('faq_questions', conn, if_exists='append', index=False)
    test_data['faq_answers'].to_sql('faq_answers', conn, if_exists='append', index=False)
    
    log_success(f"Inserted {len(test_data['content_repo'])} content records")
    log_success(f"Inserted {len(test_data['faq_questions'])} FAQ questions")
    log_success(f"Inserted {len(test_data['faq_answers'])} FAQ answers")

✅ Inserted 3 content records
✅ Inserted 10 FAQ questions
✅ Inserted 10 FAQ answers


## Step 3.5: Test Foreign Key Constraints

In [9]:
# Test that foreign key constraints are properly enforced
import json
from datetime import datetime

with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    print("🧪 Testing Foreign Key Constraints...")
    tests_passed = 0
    tests_total = 2
    
    # Test 1: Try to insert content_change_log with non-existent content_id (v4 schema)
    try:
        conn.execute("""
            INSERT INTO content_change_log (
                content_id,
                content_checksum,
                file_name,
                requires_faq_regeneration,
                source_modified_at,
                detected_at
            ) VALUES (?, ?, ?, ?, ?, datetime('now'))
        """, (
            99999,                    # Invalid content_id (should trigger FK error)
            'e' * 64,                 # Valid 64-char checksum
            'test.pdf',               # file_name
            1,                        # requires_faq_regeneration (BOOLEAN 0/1)
            datetime.now().isoformat() + 'Z'  # source_modified_at (ISO-8601 UTC)
        ))
        conn.rollback()
        log_warning("  ❌ Test 1 FAILED: Insert with invalid content_id succeeded (FK not enforced!)")
    except sqlite3.IntegrityError as e:
        log_success(f"  ✅ Test 1 PASSED: content_change_log FK constraint working")
        tests_passed += 1
    
    # Test 2: Try to insert faq_content_map with non-existent question_id (v4 schema with JSON)
    try:
        # Build metadata JSON
        current_meta = json.dumps({"current_page": 1})
        original_meta = json.dumps({"original_page": 1})
        
        conn.execute("""
            INSERT INTO faq_content_map (
                map_id,
                question_id,
                answer_id,
                content_checksum,
                current_content_id,
                current_file_name,
                current_metadata,
                original_content_id,
                original_metadata,
                created_for_version,
                is_valid,
                created_at,
                valid_from
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'), datetime('now'))
        """, (
            9999,           # map_id
            99999,          # Invalid question_id (should trigger FK error)
            99999,          # Invalid answer_id
            'f' * 64,       # content_checksum
            1001,           # current_content_id
            'test.pdf',     # current_file_name
            current_meta,   # current_metadata (JSON)
            1001,           # original_content_id
            original_meta,  # original_metadata (JSON)
            1,              # created_for_version
            1               # is_valid
        ))
        conn.rollback()
        log_warning("  ❌ Test 2 FAILED: Insert with invalid question_id succeeded (FK not enforced!)")
    except sqlite3.IntegrityError as e:
        log_success(f"  ✅ Test 2 PASSED: faq_content_map FK constraint working")
        tests_passed += 1
    
    if tests_passed == tests_total:
        log_success(f"\n✅ All {tests_total} FK constraint tests PASSED")
    else:
        log_warning(f"\n⚠️  Only {tests_passed}/{tests_total} FK constraint tests passed")

🧪 Testing Foreign Key Constraints...
✅   ✅ Test 1 PASSED: content_change_log FK constraint working
✅   ✅ Test 2 PASSED: faq_content_map FK constraint working
✅ 
✅ All 2 FK constraint tests PASSED


## Step 4: Create FAQ-Content Mappings

**Mapping Strategy:** Uses the `src_id` and `src_page_number` fields from `faq_questions` to establish the relationship between FAQs and content pages.

- Each FAQ question contains a `src_id` that references `content_repo.ud_source_file_id`
- This ensures ALL questions are mapped to their source content (not just an arbitrary subset)
- Multiple FAQs can map to the same content page (many-to-one relationship)

In [10]:
with sqlite3.connect(DB_PATH) as conn:
    # Get all FAQs with their source content references
    faqs = conn.execute("""
        SELECT q.question_id, a.answer_id, q.src_id, q.src_page_number
        FROM faq_questions q 
        LEFT JOIN faq_answers a ON q.question_id = a.question_id
        ORDER BY q.question_id
    """).fetchall()
    
    # Create mappings based on src_id relationship
    map_id = 1
    mapped_count = 0
    skipped_count = 0
    
    for question_id, answer_id, src_id, src_page in faqs:
        # Find matching content by src_id and page number
        content = conn.execute("""
            SELECT content_checksum, raw_file_nme, raw_file_page_nbr, ud_source_file_id
            FROM content_repo 
            WHERE ud_source_file_id = ? AND raw_file_page_nbr = ?
        """, (src_id, src_page)).fetchone()
        
        if content:
            checksum, file_name, page_num, content_id = content
            
            # Build JSON metadata for current and original locations (v4 schema)
            current_meta = json.dumps({
                "current_page": page_num
            })
            original_meta = json.dumps({
                "original_page": page_num
            })
            
            conn.execute("""
                INSERT INTO faq_content_map (
                    map_id,
                    question_id,
                    answer_id,
                    content_checksum,
                    current_content_id,
                    current_file_name,
                    current_metadata,
                    original_content_id,
                    original_metadata,
                    created_for_version,
                    is_valid,
                    created_at,
                    valid_from
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'), datetime('now'))
            """, (
                map_id,         # map_id
                question_id,    # question_id
                answer_id,      # answer_id
                checksum,       # content_checksum
                content_id,     # current_content_id
                file_name,      # current_file_name
                current_meta,   # current_metadata (JSON)
                content_id,     # original_content_id
                original_meta,  # original_metadata (JSON)
                1,              # created_for_version
                1               # is_valid
            ))
            map_id += 1
            mapped_count += 1
        else:
            log_warning(f"  ⚠️  Question {question_id} references non-existent content (src_id={src_id}, page={src_page})")
            skipped_count += 1
    
    conn.commit()
    count = conn.execute("SELECT COUNT(*) FROM faq_content_map").fetchone()[0]
    log_success(f"Created {count} FAQ-content mappings ({mapped_count} mapped, {skipped_count} skipped)")

✅ Created 10 FAQ-content mappings (10 mapped, 0 skipped)


## Step 5: Verify Setup

In [11]:
with sqlite3.connect(DB_PATH) as conn:
    print("\n📋 Final Database State:")
    for table in ['content_repo', 'faq_questions', 'faq_answers', 'faq_content_map', 'content_change_log']:
        count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"  {table:25} {count:3} records")
    
    # Test foreign keys
    fk_check = pd.read_sql("PRAGMA foreign_key_check", conn)
    if fk_check.empty:
        log_success("\n✅ All foreign key constraints valid")
    else:
        log_warning(f"\n⚠️  Foreign key violations found: {len(fk_check)}")

print("\n" + "="*80)
log_success("DATABASE SETUP COMPLETE")
print("="*80)


📋 Final Database State:
  content_repo                3 records
  faq_questions              10 records
  faq_answers                10 records
  faq_content_map            10 records
  content_change_log          0 records
✅ 
✅ All foreign key constraints valid

✅ DATABASE SETUP COMPLETE
